# PawCore Snowflake Intelligence Setup

**Complete setup notebook for the PawCore Intelligence mystery demo**

Run cells in order to create the environment for solving PawCore's Q4 revenue mystery using Snowflake Intelligence features.


## 🕵️ The Mystery

**PawCore's Q4 revenue dropped in EMEA for SmartCollar products. Returns surged due to faulty batteries, but were under-reported by an overwhelmed support analyst.**

**Your Mission:** Use Snowflake Intelligence to solve this mystery end-to-end.

**What You'll Learn:**
- 🔍 Semantic Views + Cortex Analyst
- 🤖 AI in SQL (AI_EXTRACT, AI_CLASSIFY, AI_COMPLETE)
- 📄 Cortex Search for unstructured data
- 🚀 Intelligent Agents with custom tools
- 📧 Automated executive reporting


In [ ]:
-- Environment Setup
USE ROLE ACCOUNTADMIN;

CREATE WAREHOUSE IF NOT EXISTS PAWCORE_INTELLIGENCE_WH
  WAREHOUSE_SIZE = 'XSMALL'
  AUTO_SUSPEND = 60
  AUTO_RESUME = TRUE;

CREATE DATABASE IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO;
CREATE SCHEMA IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA;
CREATE SCHEMA IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO.DOCUMENTS;
CREATE SCHEMA IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO.AGENTS;

USE DATABASE PAWCORE_INTELLIGENCE_DEMO;
USE SCHEMA BUSINESS_DATA;
USE WAREHOUSE PAWCORE_INTELLIGENCE_WH;

SELECT 'Environment ready!' AS status;


In [ ]:
-- Create Stages and File Formats
CREATE FILE FORMAT IF NOT EXISTS CSV_FORMAT
  TYPE = CSV
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  SKIP_HEADER = 1
  NULL_IF = ('', 'NULL')
  TRIM_SPACE = TRUE;

CREATE STAGE IF NOT EXISTS INTERNAL_DATA_STAGE FILE_FORMAT = CSV_FORMAT;
CREATE STAGE IF NOT EXISTS DOCUMENT_STAGE;
CREATE STAGE IF NOT EXISTS AUDIO_STAGE;

ALTER STAGE DOCUMENT_STAGE SET DIRECTORY = (ENABLE = TRUE);
ALTER STAGE AUDIO_STAGE SET DIRECTORY = (ENABLE = TRUE);

SELECT 'Stages created!' AS status;


## ⬆️ UPLOAD FILES NOW

**Before continuing, upload these files via Snowflake UI:**

### To INTERNAL_DATA_STAGE:
- `pawcore_sales.csv`
- `returns.csv` ⭐ **Critical for mystery**
- `hr_resumes.csv`
- `device_sales_by_region.csv`
- `pet_owners.csv`
- `vet_feedback.csv`
- `fitbit_inventory.csv`
- `email_campaigns.csv`
- `enhanced_sales_data.csv`
- `social_media_posts.csv`
- `support_tickets.csv`
- `warranty_costs.csv`

### To DOCUMENT_STAGE:
- `customer_reviews.csv` ⭐ **Critical for mystery**
- `pawcore_slack.csv` ⭐ **Critical for mystery**
- `Q4_2024_PawCore_Financial_Report.md`
- `Sales_Performance_Q4_2024.md`
- `Quarterly_Sales_Speech_PawCore.md`

### To AUDIO_STAGE:
- `PawCore Quarterly Call.mp3`

**📍 How to upload:** Go to Snowflake UI → Data → Databases → PAWCORE_INTELLIGENCE_DEMO → BUSINESS_DATA → click on stage name → "Upload Files"


In [ ]:
-- Create Tables
CREATE TABLE IF NOT EXISTS PAWCORE_SALES (
  DATE DATE,
  REGION STRING,
  PRODUCT STRING,
  FORECAST_SALES NUMBER(10,2),
  ACTUAL_SALES NUMBER(10,2),
  VARIANCE NUMBER(10,2),
  PCT_OF_FORECAST NUMBER(5,2),
  INVENTORY_UNITS_AVAILABLE INTEGER,
  MARKETING_ENGAGEMENT_SCORE INTEGER
);

-- Critical mystery table
CREATE TABLE IF NOT EXISTS RETURNS (
  RETURN_ID STRING,
  DATE DATE,
  REGION STRING,
  PRODUCT STRING,
  LOT_ID STRING,
  REASON STRING,
  QTY NUMBER,
  UNIT_COST_EUR NUMBER(10,2),
  TOTAL_COST_EUR NUMBER(10,2)
);

-- HR resumes for hiring bonus
CREATE TABLE IF NOT EXISTS HR_RESUMES (
  CANDIDATE_NAME STRING,
  RESUME_TEXT TEXT,
  APPLICATION_DATE DATE,
  POSITION_APPLIED STRING
);

-- Supporting tables
CREATE TABLE IF NOT EXISTS PET_OWNERS (
  CUSTOMER_ID STRING,
  CUSTOMER_NAME STRING,
  PET_TYPE STRING,
  PET_NAME STRING,
  REGION STRING,
  SEGMENT STRING,
  JOIN_DATE DATE
);

CREATE TABLE IF NOT EXISTS DEVICE_SALES_BY_REGION (
  DATE DATE,
  REGION STRING,
  DEVICE_TYPE STRING,
  UNITS_SOLD INTEGER,
  REVENUE NUMBER(10,2),
  GROWTH_RATE NUMBER(5,2)
);

-- Unstructured data tables
CREATE TABLE IF NOT EXISTS SLACK_MESSAGES (
  MESSAGE_ID STRING,
  CHANNEL STRING,
  AUTHOR STRING,
  MESSAGE_TEXT STRING,
  TIMESTAMP TIMESTAMP,
  SENTIMENT STRING
);

CREATE TABLE IF NOT EXISTS CUSTOMER_REVIEWS (
  REVIEW_ID STRING,
  DATE DATE,
  PRODUCT STRING,
  CUSTOMER_NAME STRING,
  RATING INTEGER,
  REVIEW_TEXT STRING,
  SENTIMENT STRING,
  REVIEW_CATEGORY STRING,
  VERIFIED_PURCHASE STRING,
  HELPFUL_VOTES INTEGER,
  REVIEW_LENGTH INTEGER,
  REGION STRING,
  CUSTOMER_SEGMENT STRING
);

-- Additional mystery support tables
CREATE TABLE IF NOT EXISTS SUPPORT_TICKETS (
  TICKET_ID STRING,
  DATE DATE,
  REGION STRING,
  PRODUCT STRING,
  FIRMWARE_VERSION STRING,
  CATEGORY STRING,
  STATUS STRING
);

CREATE TABLE IF NOT EXISTS WARRANTY_COSTS (
  PERIOD STRING,
  REGION STRING,
  PRODUCT STRING,
  COST_EUR NUMBER(10,2)
);

SELECT 'Tables created!' AS status;


In [ ]:
-- Load Critical Data (Sales, Returns, HR Resumes)
COPY INTO PAWCORE_SALES (
  DATE, REGION, PRODUCT, FORECAST_SALES, ACTUAL_SALES, VARIANCE,
  PCT_OF_FORECAST, INVENTORY_UNITS_AVAILABLE, MARKETING_ENGAGEMENT_SCORE
)
FROM @INTERNAL_DATA_STAGE/pawcore_sales.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load critical returns data for mystery
COPY INTO RETURNS (
  RETURN_ID, DATE, REGION, PRODUCT, LOT_ID, REASON, QTY, UNIT_COST_EUR, TOTAL_COST_EUR
)
FROM @INTERNAL_DATA_STAGE/returns.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load HR resumes for hiring bonus
COPY INTO HR_RESUMES (
  CANDIDATE_NAME, RESUME_TEXT, APPLICATION_DATE, POSITION_APPLIED
)
FROM @INTERNAL_DATA_STAGE/hr_resumes.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

SELECT 'Critical data loaded!' AS status;


In [ ]:
-- Load Unstructured Data (Slack & Reviews)

-- Slack messages with staging
CREATE OR REPLACE TABLE SLACK_MESSAGES_STG (URL STRING, CHANNEL STRING, TS STRING, TEXT STRING);
COPY INTO SLACK_MESSAGES_STG FROM @DOCUMENT_STAGE/pawcore_slack.csv FILE_FORMAT = CSV_FORMAT FORCE = TRUE ON_ERROR = 'CONTINUE';

INSERT INTO SLACK_MESSAGES (MESSAGE_ID, CHANNEL, AUTHOR, MESSAGE_TEXT, TIMESTAMP, SENTIMENT)
SELECT TS, CHANNEL, NULL, TEXT, TO_TIMESTAMP_NTZ(TRY_TO_NUMBER(SUBSTR(TS,1,10))), NULL
FROM SLACK_MESSAGES_STG;

-- Add sentiment analysis
UPDATE SLACK_MESSAGES
SET SENTIMENT = SNOWFLAKE.CORTEX.SENTIMENT(MESSAGE_TEXT)
WHERE SENTIMENT IS NULL;

-- Customer reviews
COPY INTO CUSTOMER_REVIEWS
FROM @DOCUMENT_STAGE/customer_reviews.csv
FILE_FORMAT = CSV_FORMAT
FORCE = TRUE
ON_ERROR = 'CONTINUE';

-- Quick supporting data load
CREATE OR REPLACE TABLE PET_OWNERS_STG (CUSTOMER_NAME STRING, PHONE STRING, EMAIL STRING);
COPY INTO PET_OWNERS_STG FROM @INTERNAL_DATA_STAGE/pet_owners.csv FILE_FORMAT = CSV_FORMAT ON_ERROR = 'CONTINUE';
INSERT INTO PET_OWNERS (CUSTOMER_ID, CUSTOMER_NAME, PET_TYPE, PET_NAME, REGION, SEGMENT, JOIN_DATE)
SELECT UUID_STRING(), CUSTOMER_NAME, NULL, NULL, NULL, NULL, CURRENT_DATE() FROM PET_OWNERS_STG;

-- Load additional mystery support data
COPY INTO SUPPORT_TICKETS (TICKET_ID, DATE, REGION, PRODUCT, FIRMWARE_VERSION, CATEGORY, STATUS)
FROM @INTERNAL_DATA_STAGE/support_tickets.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

COPY INTO WARRANTY_COSTS (PERIOD, REGION, PRODUCT, COST_EUR)
FROM @INTERNAL_DATA_STAGE/warranty_costs.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

SELECT 'All data loaded!' AS status;


In [ ]:
-- Verify Data Load
SELECT 'PAWCORE_SALES' AS table_name, COUNT(*) AS row_count FROM PAWCORE_SALES
UNION ALL SELECT 'RETURNS', COUNT(*) FROM RETURNS  -- Should be ~9 rows
UNION ALL SELECT 'HR_RESUMES', COUNT(*) FROM HR_RESUMES  -- Should be 8 rows
UNION ALL SELECT 'CUSTOMER_REVIEWS', COUNT(*) FROM CUSTOMER_REVIEWS
UNION ALL SELECT 'SLACK_MESSAGES', COUNT(*) FROM SLACK_MESSAGES
UNION ALL SELECT 'PET_OWNERS', COUNT(*) FROM PET_OWNERS
UNION ALL SELECT 'SUPPORT_TICKETS', COUNT(*) FROM SUPPORT_TICKETS
UNION ALL SELECT 'WARRANTY_COSTS', COUNT(*) FROM WARRANTY_COSTS
ORDER BY table_name;


In [ ]:
-- Mystery Preview: Spot the Anomaly!
WITH weekly_returns AS (
    SELECT 
        DATE_TRUNC('WEEK', DATE) AS week_start,
        LOT_ID,
        SUM(QTY) AS returned_units
    FROM RETURNS
    WHERE PRODUCT = 'SmartCollar' AND REGION = 'EMEA'
    GROUP BY 1, 2
),
baseline AS (
    SELECT AVG(returned_units) AS avg_baseline
    FROM weekly_returns
    WHERE week_start < '2024-10-01'
)
SELECT 
    w.week_start,
    w.LOT_ID,
    w.returned_units,
    ROUND(b.avg_baseline, 1) AS baseline,
    ROUND(w.returned_units / b.avg_baseline, 1) AS spike_multiplier,
    CASE 
        WHEN w.returned_units >= 3 * b.avg_baseline THEN '🚨 FLAGGED'
        ELSE 'Normal'
    END AS status
FROM weekly_returns w
CROSS JOIN baseline b
WHERE w.week_start >= '2024-10-01'
ORDER BY w.week_start, w.LOT_ID;


## 🎉 Setup Complete!

**Your PawCore mystery environment is ready!**

**Next Steps:**
1. 📖 Follow the **Master Guide**: `COMPLETE_MASTER_HOL_GUIDE.md`
2. 🕵️ Start with **Phase 1**: Detect the Gap using Cortex Analyst
3. 🔍 Create **Semantic Views** for business-friendly querying
4. 🤖 Use **AI in SQL** functions for deeper analysis
5. 📧 Build **Intelligent Agents** for automated reporting

**Key Mystery Data:**
- `RETURNS`: Look for Lot 341 anomalies ⭐
- `CUSTOMER_REVIEWS`: Q4 SmartCollar complaints ⭐
- `SLACK_MESSAGES`: Support escalation patterns ⭐
- `SUPPORT_TICKETS`: Evidence of support team overwhelm ⭐
- `WARRANTY_COSTS`: Financial impact escalation ⭐
- `HR_RESUMES`: Candidates for staffing solution

**Ready to solve the mystery? Let's go! 🕵️‍♂️**


# PawCore Intelligence Demo Notebook
Consolidated end-to-end setup for the PawCore Intelligence demo in Snowflake. Run cells in order. Headings indicate each section (0–8).

- Role used for setup: `ACCOUNTADMIN` (or `SNOWFLAKE_INTELLIGENCE_ADMIN_RL` if available)
- Warehouse: `PAWCORE_INTELLIGENCE_WH`
- Database/Schemas: `PAWCORE_INTELLIGENCE_DEMO` (`BUSINESS_DATA`, `DOCUMENTS`, `AGENTS`)

### Story: The Mystery of the Missing Revenue
Q4 revenue beat targets overall by 15%, but EMEA under‑performed by 23% for SmartCollar. Marketing spent 40% more in EMEA, CSAT dropped 15%, and battery‑related support tickets spiked 200%. We’ll unify structured + unstructured signals to find root cause and prevent future losses.

### Demo dataset
- Download link: [Open the demo data in Google Drive](https://drive.google.com/file/d/1c86Tx06ECshDkC2pdqI59r71FUZEfvPf/view?usp=drive_link)
- Permissions: You can run this as `ACCOUNTADMIN`. Optionally, create and use `PAWCORE_DEMO_ROLE` (least‑privilege) in Section 0a and switch in the "switch_to_demo_role_optional" cell later.
- After you run Sections 0 and 1 (which create the stages), upload the files using the "Upload files now" instructions below, then run the quick checks.

### Assumptions
- You can sign in to the Snowflake UI (Snowsight)
- You have `ACCOUNTADMIN` (or equivalent) to create warehouses, databases, schemas, stages, and file formats
- You can upload files to internal stages from Snowsight (or via SnowSQL `PUT` if preferred)
- Network access allows browser uploads to Snowflake
- You have the demo ZIP downloaded locally

In [ ]:
-- 0) Environment setup (one-time, safe re-creates)

USE ROLE ACCOUNTADMIN;
CREATE WAREHOUSE IF NOT EXISTS PAWCORE_INTELLIGENCE_WH
  WAREHOUSE_SIZE = 'XSMALL' AUTO_SUSPEND = 60 AUTO_RESUME = TRUE;

CREATE DATABASE IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO;
CREATE SCHEMA IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA;
CREATE SCHEMA IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO.DOCUMENTS;
CREATE SCHEMA IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO.AGENTS;

USE DATABASE PAWCORE_INTELLIGENCE_DEMO;
USE SCHEMA   BUSINESS_DATA;
USE WAREHOUSE PAWCORE_INTELLIGENCE_WH;

### 0a) Optional: Minimal demo role
Creates a lightweight role for demo users with the least privileges to run this notebook and build semantic views/agents.

### ACTION REQUIRED: set your Snowflake username
Before running the next cell, update the `GRANT ROLE ... TO USER ...;` statement with your Snowflake username (replace any placeholder).

To confirm your username, run:
```sql
SELECT CURRENT_USER();
```


In [ ]:
-- To confirm your username, run:
SELECT CURRENT_USER();

In [ ]:
-- 0a) (Optional) Create a minimal demo role for customers
CREATE ROLE IF NOT EXISTS PAWCORE_DEMO_ROLE;
-- Replace <your_user> with your Snowflake username:
GRANT ROLE PAWCORE_DEMO_ROLE TO USER CALEXANDER;

-- Warehouse access
GRANT USAGE ON WAREHOUSE PAWCORE_INTELLIGENCE_WH TO ROLE PAWCORE_DEMO_ROLE;

-- Data access and build privileges on BUSINESS_DATA
GRANT USAGE ON DATABASE PAWCORE_INTELLIGENCE_DEMO TO ROLE PAWCORE_DEMO_ROLE;
GRANT USAGE ON SCHEMA  PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA TO ROLE PAWCORE_DEMO_ROLE;
GRANT CREATE TABLE, CREATE STAGE, CREATE FILE FORMAT, CREATE SEMANTIC VIEW
  ON SCHEMA PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA TO ROLE PAWCORE_DEMO_ROLE;

-- Table DML
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA TO ROLE PAWCORE_DEMO_ROLE;
GRANT SELECT, INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA TO ROLE PAWCORE_DEMO_ROLE;

-- Stage and file format usage
GRANT USAGE ON ALL STAGES    IN SCHEMA PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA TO ROLE PAWCORE_DEMO_ROLE;
GRANT USAGE ON FUTURE STAGES IN SCHEMA PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA TO ROLE PAWCORE_DEMO_ROLE;
GRANT USAGE ON ALL FILE FORMATS IN SCHEMA PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA TO ROLE PAWCORE_DEMO_ROLE;
GRANT USAGE ON FUTURE FILE FORMATS IN SCHEMA PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA TO ROLE PAWCORE_DEMO_ROLE;

-- Agent (optional)
GRANT USAGE ON SCHEMA PAWCORE_INTELLIGENCE_DEMO.AGENTS TO ROLE PAWCORE_DEMO_ROLE;
GRANT CREATE AGENT ON SCHEMA PAWCORE_INTELLIGENCE_DEMO.AGENTS TO ROLE PAWCORE_DEMO_ROLE;

-- Notebook editor permission for Streamlit preview is configured in Snowsight UI (Share/Permissions) and cannot be granted via SQL.
-- Run Streamlit cells as the notebook owner role, or duplicate the notebook under your role, or add your role as Editor in Snowsight.
-- Do not switch roles here. Use the 'switch_to_demo_role_optional' cell later if desired.

## 1) File format and stages
Defines a reusable CSV file format and creates internal stages for structured and unstructured data (documents/images/audio). Enables directory tables for listing staged files.

In [ ]:
-- Ensure context before creating objects
USE ROLE ACCOUNTADMIN;
USE DATABASE PAWCORE_INTELLIGENCE_DEMO;
USE SCHEMA BUSINESS_DATA;
USE WAREHOUSE PAWCORE_INTELLIGENCE_WH;


In [ ]:
-- 1) File format + stages (fully-qualified names)
CREATE FILE FORMAT IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.CSV_FORMAT 
  TYPE = CSV 
  FIELD_OPTIONALLY_ENCLOSED_BY = '"' 
  SKIP_HEADER = 1
  NULL_IF = ('', 'NULL')
  TRIM_SPACE = TRUE;

CREATE STAGE IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.INTERNAL_DATA_STAGE 
  FILE_FORMAT = PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.CSV_FORMAT;
CREATE STAGE IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.DOCUMENT_STAGE;
CREATE STAGE IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.IMAGE_STAGE;
CREATE STAGE IF NOT EXISTS PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.AUDIO_STAGE;

-- Enable directory tables
ALTER STAGE PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.DOCUMENT_STAGE SET DIRECTORY = (ENABLE = TRUE);
ALTER STAGE PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.IMAGE_STAGE     SET DIRECTORY = (ENABLE = TRUE);
ALTER STAGE PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.AUDIO_STAGE     SET DIRECTORY = (ENABLE = TRUE);
ALTER STAGE PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.INTERNAL_DATA_STAGE SET DIRECTORY = (ENABLE = TRUE);

-- Default file format on internal stage
ALTER STAGE PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.INTERNAL_DATA_STAGE 
  SET FILE_FORMAT = PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.CSV_FORMAT;


In [ ]:
### Upload files now
Note: Do not end or leave this notebook. Open a new browser tab, go to Snowsight → Data → Databases → `PAWCORE_INTELLIGENCE_DEMO` → `BUSINESS_DATA` → Stages, upload the files there, then return to this tab and run the quick checks below.

- Internal Data Stage (`@INTERNAL_DATA_STAGE`)
  - `pawcore_sales.csv`, `pet_owners.csv`, `device_sales_by_region.csv`, `fitbit_inventory.csv`, `vet_feedback.csv`, `email_campaigns.csv`, `enhanced_sales_data.csv`, `social_media_posts.csv`
- Document Stage (`@DOCUMENT_STAGE`)
  - Required: `customer_reviews.csv`, `pawcore_slack.csv`
  - Optional: `Q4_2024_PawCore_Financial_Report.md`, `Quarterly_Sales_Speech_PawCore.md`, `Sales_Performance_Q4_2024.md`
- Image Stage (`@IMAGE_STAGE`)
  - `SmartCollar Product Photo.jpeg`, `Customer Journey Infographic.pdf`, `barkour.jpg`
- Audio Stage (`@AUDIO_STAGE`)
  - `PawCore Quarterly Call.mp3`

Expected totals: Internal=8, Document=5 (2+3), Image=3, Audio=1.


## 2) Quick directory checks (optional)
Utility commands to verify files are present in stages. Uncomment the commands you want to run.

In [ ]:
-- 2) Quick directory checks (fast counts; no refresh)
USE DATABASE PAWCORE_INTELLIGENCE_DEMO;
USE SCHEMA BUSINESS_DATA;

WITH counts AS (
  SELECT 'INTERNAL_DATA_STAGE' AS stage_name,
         COUNT(*) AS num_files,
         MIN(relative_path) AS sample_file
  FROM DIRECTORY(@PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.INTERNAL_DATA_STAGE)
  UNION ALL
  SELECT 'DOCUMENT_STAGE', COUNT(*), MIN(relative_path)
  FROM DIRECTORY(@PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.DOCUMENT_STAGE)
  UNION ALL
  SELECT 'IMAGE_STAGE', COUNT(*), MIN(relative_path)
  FROM DIRECTORY(@PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.IMAGE_STAGE)
  UNION ALL
  SELECT 'AUDIO_STAGE', COUNT(*), MIN(relative_path)
  FROM DIRECTORY(@PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.AUDIO_STAGE)
)
SELECT stage_name,
       num_files,
       IFF(num_files > 0, 'PASS', 'WARN') AS status,
       COALESCE(sample_file, '') AS sample_file
FROM counts
ORDER BY stage_name;

### Optional: Switch to the minimal demo role
If you created `PAWCORE_DEMO_ROLE`, you can run the rest of the notebook with least-privilege access. Skip this if you are the admin running the setup.



In [ ]:
-- Switch to demo role (optional)
USE ROLE PAWCORE_DEMO_ROLE;
USE DATABASE PAWCORE_INTELLIGENCE_DEMO;
USE SCHEMA BUSINESS_DATA;
USE WAREHOUSE PAWCORE_INTELLIGENCE_WH;


## 3) Structured tables
Creates core relational tables used across sales, customers, devices, inventory, feedback, email campaigns, enhanced sales, and social posts.

In [ ]:
-- 3) Structured tables
CREATE TABLE IF NOT EXISTS PAWCORE_SALES (
  DATE DATE, REGION STRING, PRODUCT STRING,
  FORECAST_SALES NUMBER(10,2), ACTUAL_SALES NUMBER(10,2), VARIANCE NUMBER(10,2),
  PCT_OF_FORECAST NUMBER(5,2), INVENTORY_UNITS_AVAILABLE INTEGER, MARKETING_ENGAGEMENT_SCORE INTEGER
);
CREATE TABLE IF NOT EXISTS PET_OWNERS (
  CUSTOMER_ID STRING, CUSTOMER_NAME STRING, PET_TYPE STRING, PET_NAME STRING,
  REGION STRING, SEGMENT STRING, JOIN_DATE DATE
);
CREATE TABLE IF NOT EXISTS DEVICE_SALES_BY_REGION (
  DATE DATE, REGION STRING, DEVICE_TYPE STRING, UNITS_SOLD INTEGER,
  REVENUE NUMBER(10,2), GROWTH_RATE NUMBER(5,2)
);
CREATE TABLE IF NOT EXISTS FITBIT_INVENTORY (
  PRODUCT_ID STRING, PRODUCT_NAME STRING, REGION STRING,
  INVENTORY_LEVEL INTEGER, REORDER_POINT INTEGER, LAST_RESTOCK_DATE DATE
);
CREATE TABLE IF NOT EXISTS VET_FEEDBACK (
  FEEDBACK_ID STRING, VET_NAME STRING, PRODUCT STRING, RATING INTEGER,
  FEEDBACK_TEXT STRING, DATE DATE, REGION STRING
);
CREATE TABLE IF NOT EXISTS EMAIL_CAMPAIGNS (
  CAMPAIGN_ID STRING, CAMPAIGN_NAME STRING, START_DATE DATE, END_DATE DATE,
  REGION STRING, BUDGET NUMBER(10,2), SPEND NUMBER(10,2), CLICKS INTEGER, CONVERSIONS INTEGER
);
CREATE TABLE IF NOT EXISTS ENHANCED_SALES_DATA (
  SALE_ID STRING, DATE DATE, REGION STRING, PRODUCT STRING, CUSTOMER_SEGMENT STRING,
  SALES_AMOUNT NUMBER(10,2), DISCOUNT_AMOUNT NUMBER(10,2), NET_SALES NUMBER(10,2)
);
CREATE TABLE IF NOT EXISTS SOCIAL_MEDIA_POSTS (
  POST_ID STRING, PLATFORM STRING, POST_DATE DATE, CONTENT STRING,
  ENGAGEMENT_SCORE INTEGER, SENTIMENT STRING, REGION STRING
);

### Action required: upload structured CSVs to the internal stage
Before running the next cell, upload the structured CSV files to `@INTERNAL_DATA_STAGE` (e.g., `pawcore_sales.csv`, `pet_owners.csv`, `device_sales_by_region.csv`, `fitbit_inventory.csv`, `vet_feedback.csv`, `email_campaigns.csv`, `enhanced_sales_data.csv`, `social_media_posts.csv`).

You can use Snowsight’s Stage file upload, or run `PUT` commands from SnowSQL. Verify with:
```sql
LIST @PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.INTERNAL_DATA_STAGE;
```


In [ ]:
LIST @PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.INTERNAL_DATA_STAGE;

### Check and reset context (run if you see “object does not exist” errors)
This cell shows your current context and re-asserts the expected database/schema/warehouse for the demo.


In [ ]:
-- -- Context diagnostics
-- SELECT CURRENT_ROLE()   AS ROLE,
--        CURRENT_DATABASE() AS DB,
--        CURRENT_SCHEMA()   AS SCH,
--        CURRENT_WAREHOUSE() AS WH;

-- -- Re-assert expected context (safe to re-run)
-- USE ROLE ACCOUNTADMIN;
-- USE DATABASE PAWCORE_INTELLIGENCE_DEMO;
-- USE SCHEMA BUSINESS_DATA;
-- USE WAREHOUSE PAWCORE_INTELLIGENCE_WH;

## 4) Structured loads
Loads CSVs from the internal stage into the structured tables, using staging tables where needed and performing type coercion and simple cleansing.

In [ ]:
-- 4) Structured loads
COPY INTO PAWCORE_SALES (DATE, REGION, PRODUCT, FORECAST_SALES, ACTUAL_SALES, VARIANCE, PCT_OF_FORECAST, INVENTORY_UNITS_AVAILABLE, MARKETING_ENGAGEMENT_SCORE)
FROM @INTERNAL_DATA_STAGE/pawcore_sales.csv FILE_FORMAT = CSV_FORMAT ON_ERROR = 'CONTINUE';

CREATE OR REPLACE TABLE PET_OWNERS_STG (CUSTOMER_NAME STRING, PHONE STRING, EMAIL STRING);
TRUNCATE TABLE PET_OWNERS_STG;
COPY INTO PET_OWNERS_STG FROM @INTERNAL_DATA_STAGE/pet_owners.csv FILE_FORMAT = CSV_FORMAT FORCE = TRUE ON_ERROR='CONTINUE';
INSERT INTO PET_OWNERS (CUSTOMER_ID, CUSTOMER_NAME, PET_TYPE, PET_NAME, REGION, SEGMENT, JOIN_DATE)
SELECT UUID_STRING(), CUSTOMER_NAME, NULL, NULL, NULL, NULL, CURRENT_DATE() FROM PET_OWNERS_STG;

CREATE OR REPLACE TABLE DEVICE_SALES_BY_REGION_STG AS
SELECT $1::STRING AS C1,$2::STRING AS C2,$3::STRING AS C3,$4::STRING AS C4,$5::STRING AS C5,$6::STRING AS C6
FROM @INTERNAL_DATA_STAGE/device_sales_by_region.csv (FILE_FORMAT=>'CSV_FORMAT');
TRUNCATE TABLE DEVICE_SALES_BY_REGION;
INSERT INTO DEVICE_SALES_BY_REGION (DATE, REGION, DEVICE_TYPE, UNITS_SOLD, REVENUE, GROWTH_RATE)
SELECT COALESCE(TRY_TO_DATE(s.C1), TRY_TO_DATE(s.C6), TO_DATE('2024-01-01')),
       CASE WHEN TRY_TO_DATE(s.C1) IS NULL THEN s.C1 ELSE s.C2 END,
       CASE WHEN TRY_TO_DATE(s.C1) IS NULL THEN s.C2 ELSE s.C3 END,
       TRY_TO_NUMBER(CASE WHEN TRY_TO_DATE(s.C1) IS NULL THEN s.C3 ELSE s.C4 END),
       TRY_TO_NUMBER(REPLACE(REPLACE(CASE WHEN TRY_TO_DATE(s.C1) IS NULL THEN s.C4 ELSE s.C5 END,'$',''),',','')),
       TRY_TO_NUMBER(REPLACE(CASE WHEN TRY_TO_DATE(s.C1) IS NULL THEN s.C5 ELSE s.C6 END,'%',''))/100
FROM DEVICE_SALES_BY_REGION_STG s;

CREATE OR REPLACE TABLE FITBIT_INVENTORY_STG AS
SELECT $1::STRING PRODUCT_ID,$2::STRING PRODUCT_NAME,$3::STRING REGION,$4::STRING INVENTORY_LEVEL,$5::STRING REORDER_POINT,$6::STRING LAST_RESTOCK_DATE
FROM @INTERNAL_DATA_STAGE/fitbit_inventory.csv (FILE_FORMAT=>'CSV_FORMAT');
INSERT INTO FITBIT_INVENTORY (PRODUCT_ID, PRODUCT_NAME, REGION, INVENTORY_LEVEL, REORDER_POINT, LAST_RESTOCK_DATE)
SELECT PRODUCT_ID, PRODUCT_NAME, REGION, TRY_TO_NUMBER(INVENTORY_LEVEL)::INT, TRY_TO_NUMBER(REORDER_POINT)::INT, TRY_TO_DATE(LAST_RESTOCK_DATE)
FROM FITBIT_INVENTORY_STG;

CREATE OR REPLACE TABLE VET_FEEDBACK_STG AS
SELECT $1::STRING FEEDBACK_ID,$2::STRING VET_NAME,$3::STRING PRODUCT,$4::STRING RATING,$5::STRING FEEDBACK_TEXT,$6::STRING DT,$7::STRING REGION
FROM @INTERNAL_DATA_STAGE/vet_feedback.csv (FILE_FORMAT=>'CSV_FORMAT');
INSERT INTO VET_FEEDBACK (FEEDBACK_ID, VET_NAME, PRODUCT, RATING, FEEDBACK_TEXT, DATE, REGION)
SELECT FEEDBACK_ID, VET_NAME, PRODUCT, TRY_TO_NUMBER(RATING)::INT, FEEDBACK_TEXT, TRY_TO_DATE(DT), REGION FROM VET_FEEDBACK_STG;

CREATE OR REPLACE TABLE EMAIL_CAMPAIGNS_STG AS
SELECT $1::STRING CAMPAIGN_ID,$2::STRING CAMPAIGN_NAME,$3::STRING START_DT,$4::STRING TAGLINE,$5::STRING REGION,$6::STRING BUDGET,$7::STRING SPEND,$8::STRING CLICKS,$9::STRING CONVERSIONS
FROM @INTERNAL_DATA_STAGE/email_campaigns.csv (FILE_FORMAT=>'CSV_FORMAT');
INSERT INTO EMAIL_CAMPAIGNS (CAMPAIGN_ID, CAMPAIGN_NAME, START_DATE, END_DATE, REGION, BUDGET, SPEND, CLICKS, CONVERSIONS)
SELECT CAMPAIGN_ID, CAMPAIGN_NAME, TRY_TO_DATE(START_DT), NULL, REGION, TRY_TO_NUMBER(BUDGET), TRY_TO_NUMBER(SPEND), TRY_TO_NUMBER(CLICKS)::INT, TRY_TO_NUMBER(CONVERSIONS)::INT
FROM EMAIL_CAMPAIGNS_STG;
UPDATE EMAIL_CAMPAIGNS SET END_DATE = DATEADD(day,30,START_DATE) WHERE END_DATE IS NULL AND START_DATE IS NOT NULL;

CREATE OR REPLACE TABLE ENHANCED_SALES_DATA_STG AS
SELECT $1::STRING SALE_ID,$2::STRING DT,$3::STRING REGION,$4::STRING PRODUCT,$5::STRING CUSTOMER_SEGMENT,$6::STRING SALES_AMOUNT,$7::STRING DISCOUNT_AMOUNT,$8::STRING NET_SALES
FROM @INTERNAL_DATA_STAGE/enhanced_sales_data.csv (FILE_FORMAT=>'CSV_FORMAT');
INSERT INTO ENHANCED_SALES_DATA (SALE_ID, DATE, REGION, PRODUCT, CUSTOMER_SEGMENT, SALES_AMOUNT, DISCOUNT_AMOUNT, NET_SALES)
SELECT SALE_ID, TRY_TO_DATE(DT), REGION, PRODUCT, CUSTOMER_SEGMENT, TRY_TO_NUMBER(SALES_AMOUNT), TRY_TO_NUMBER(DISCOUNT_AMOUNT), TRY_TO_NUMBER(NET_SALES)
FROM ENHANCED_SALES_DATA_STG;

CREATE OR REPLACE TABLE SOCIAL_MEDIA_POSTS_STG AS
SELECT $1::STRING POST_ID,$2::STRING PLATFORM,$3::STRING POST_DT,$4::STRING CONTENT,$5::STRING ENGAGEMENT_SCORE,$6::STRING SENTIMENT,$7::STRING REGION
FROM @INTERNAL_DATA_STAGE/social_media_posts.csv (FILE_FORMAT=>'CSV_FORMAT');
INSERT INTO SOCIAL_MEDIA_POSTS (POST_ID, PLATFORM, POST_DATE, CONTENT, ENGAGEMENT_SCORE, SENTIMENT, REGION)
SELECT POST_ID, PLATFORM, TRY_TO_DATE(POST_DT), CONTENT, TRY_TO_NUMBER(ENGAGEMENT_SCORE)::INT, SENTIMENT, REGION
FROM SOCIAL_MEDIA_POSTS_STG;

## Optional: Backfill sales data for 2019–2025
If your `PAWCORE_SALES` CSV only contains 2018, run the next cell to synthesize years 2019–2025 by shifting 2018 dates forward and applying small growth multipliers. Safe to re‑run; it skips rows that already exist.


In [ ]:
-- 3b) Optional backfill for 2019–2025 using 2018 as a template
-- Creates missing years by shifting dates and scaling measures slightly
WITH base_2018 AS (
  SELECT * FROM PAWCORE_SALES WHERE DATE >= '2018-01-01' AND DATE < '2019-01-01'
),
params AS (
  SELECT * FROM (
    SELECT 2019 AS year_, 1.03 AS growth UNION ALL
    SELECT 2020, 0.98 UNION ALL
    SELECT 2021, 1.04 UNION ALL
    SELECT 2022, 1.03 UNION ALL
    SELECT 2023, 1.02 UNION ALL
    SELECT 2024, 1.01 UNION ALL
    SELECT 2025, 1.03
  )
)
INSERT INTO PAWCORE_SALES (DATE, REGION, PRODUCT, FORECAST_SALES, ACTUAL_SALES, VARIANCE, PCT_OF_FORECAST, INVENTORY_UNITS_AVAILABLE, MARKETING_ENGAGEMENT_SCORE)
SELECT DATEADD(year, p.year_ - 2018, b.DATE) AS DATE,
       b.REGION,
       b.PRODUCT,
       ROUND(b.FORECAST_SALES * p.growth, 2),
       ROUND(b.ACTUAL_SALES   * p.growth, 2),
       ROUND((b.ACTUAL_SALES * p.growth) - (b.FORECAST_SALES * p.growth), 2) AS VARIANCE,
       b.PCT_OF_FORECAST,
       b.INVENTORY_UNITS_AVAILABLE,
       b.MARKETING_ENGAGEMENT_SCORE
FROM base_2018 b
JOIN params p
  ON TRUE
LEFT JOIN PAWCORE_SALES e
  ON e.DATE = DATEADD(year, p.year_ - 2018, b.DATE)
 AND e.REGION = b.REGION
 AND e.PRODUCT = b.PRODUCT
WHERE e.DATE IS NULL;


### Reminder: upload unstructured files before running
Ensure `customer_reviews.csv` and `pawcore_slack.csv` have been uploaded to `@DOCUMENT_STAGE`. Optional documents, images, and audio can also be uploaded to their respective stages for later demos.


## 5) Structured loads verification
Run the next cell to confirm row counts across all structured tables. It uses fully-qualified names so it works regardless of session context.

In [ ]:
SELECT 'PAWCORE_SALES', COUNT(*) FROM PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.PAWCORE_SALES
UNION ALL SELECT 'PET_OWNERS', COUNT(*) FROM PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.PET_OWNERS
UNION ALL SELECT 'DEVICE_SALES_BY_REGION', COUNT(*) FROM PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.DEVICE_SALES_BY_REGION
UNION ALL SELECT 'FITBIT_INVENTORY', COUNT(*) FROM PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.FITBIT_INVENTORY
UNION ALL SELECT 'VET_FEEDBACK', COUNT(*) FROM PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.VET_FEEDBACK
UNION ALL SELECT 'EMAIL_CAMPAIGNS', COUNT(*) FROM PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.EMAIL_CAMPAIGNS
UNION ALL SELECT 'ENHANCED_SALES_DATA', COUNT(*) FROM PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.ENHANCED_SALES_DATA
UNION ALL SELECT 'SOCIAL_MEDIA_POSTS', COUNT(*) FROM PAWCORE_INTELLIGENCE_DEMO.BUSINESS_DATA.SOCIAL_MEDIA_POSTS;

## 6) Unstructured data (Slack, Reviews)
Creates tables for Slack messages and customer reviews. Loads the CSVs, enriches Slack with sentiment using Cortex, and stores reviews with basic metadata.

In [ ]:
-- 6) Unstructured CSVs (Slack, Reviews)
CREATE OR REPLACE TABLE SLACK_MESSAGES (
  MESSAGE_ID STRING, CHANNEL STRING, AUTHOR STRING, MESSAGE_TEXT STRING, TIMESTAMP TIMESTAMP, SENTIMENT STRING
);
CREATE OR REPLACE TABLE CUSTOMER_REVIEWS (
  REVIEW_ID STRING, DATE DATE, PRODUCT STRING, CUSTOMER_NAME STRING, RATING INTEGER, REVIEW_TEXT STRING, SENTIMENT STRING,
  REVIEW_CATEGORY STRING, VERIFIED_PURCHASE STRING, HELPFUL_VOTES INTEGER, REVIEW_LENGTH INTEGER, REGION STRING, CUSTOMER_SEGMENT STRING
);

-- Slack load via staging (file has 4 columns)
CREATE OR REPLACE TABLE SLACK_MESSAGES_STG (
  URL STRING, CHANNEL STRING, TS STRING, TEXT STRING
);
COPY INTO SLACK_MESSAGES_STG
FROM @DOCUMENT_STAGE/pawcore_slack.csv
FILE_FORMAT=CSV_FORMAT
FORCE=TRUE
ON_ERROR='CONTINUE';

TRUNCATE TABLE SLACK_MESSAGES;
INSERT INTO SLACK_MESSAGES (MESSAGE_ID, CHANNEL, AUTHOR, MESSAGE_TEXT, TIMESTAMP, SENTIMENT)
SELECT TS,
       CHANNEL,
       NULL,
       TEXT,
       TO_TIMESTAMP_NTZ(TRY_TO_NUMBER(SUBSTR(TS,1,10))),
       NULL
FROM SLACK_MESSAGES_STG;

-- Sentiment enrichment
UPDATE SLACK_MESSAGES
SET SENTIMENT = SNOWFLAKE.CORTEX.SENTIMENT(MESSAGE_TEXT)
WHERE SENTIMENT IS NULL;

-- Customer reviews load
COPY INTO CUSTOMER_REVIEWS
FROM @DOCUMENT_STAGE/customer_reviews.csv
FILE_FORMAT=CSV_FORMAT
FORCE=TRUE
ON_ERROR='CONTINUE';

## 7) Unstructured loads verification
Row counts for Slack messages and customer reviews.

In [ ]:
-- 7) Unstructured loads verification
SELECT 'SLACK_MESSAGES', COUNT(*) FROM SLACK_MESSAGES
UNION ALL SELECT 'CUSTOMER_REVIEWS', COUNT(*) FROM CUSTOMER_REVIEWS;

## 8) Cleanup (one-step full reset)
Run this to remove everything created by this notebook and return the account to its prior state. This drops the demo database (with all schemas/tables/semantic views), the demo warehouse, and the optional demo role.

In [ ]:
-- 8) Cleanup (full reset)
-- WARNING: This will drop all demo objects created by this notebook
USE ROLE ACCOUNTADMIN;

-- Drop database (schemas, tables, stages, semantic views)
DROP DATABASE IF EXISTS PAWCORE_INTELLIGENCE_DEMO CASCADE;

-- Drop warehouse
DROP WAREHOUSE IF EXISTS PAWCORE_INTELLIGENCE_WH;

-- Drop optional demo role (if created)
DROP ROLE IF EXISTS PAWCORE_DEMO_ROLE;